# Neural Net observatory

In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib inline
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.max_open_warning'] = 0
import ipywidgets as widgets

Fetch our tools:

In [3]:
from nn import Network, Layer, IdentityLayer, AffineLayer, MapLayer
from nnbench import NNBench
from nnvis import NNVis

Use [`ipywidgets`](https://ipywidgets.readthedocs.io/en/latest/index.html)

# Multiprocessing
We run the net training in a child process, so that it can proceed while we observe and analyze partial results.

### Tooling
 * `JSONConn` over the Process Pipe
 -- Not seeing the exception on `recv()` of a closed connection, so we accomplish a close by a non-JSON message of four bytes of zero

In [4]:
from multiprocessing import Process, Pipe
import json
from time import sleep

class JSONConn():
    def __init__(self, conn):
        self.conn = conn
        
    def send(self, v):
        self.conn.send_bytes(json.dumps(v).encode('utf8'))
        
    def poll(self):
        return self.conn.poll()
    
    def recv(self):
        r = self.conn.recv_bytes()
        if r == bytes(4):
            self.close()
            raise EOFError
        return json.loads(r)
        
    def close(self):
        self.conn.send_bytes(bytes(4))
        self.conn.close()

### The child

In [5]:
def f(conn):
    jc = JSONConn(conn)
 
    net = Network()
    net.extend(AffineLayer(2,2))
    net.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d)**2))
    net.extend(AffineLayer(2,1))
    net.extend(MapLayer(np.tanh, lambda d: 1.0 - np.tanh(d)**2))

    training_batch = (np.array([[-0.5, -0.5],
                                [-0.5,  0.5],
                                [ 0.5,  0.5],
                                [ 0.5, -0.5]]),
                      np.array([[-0.5],
                                [ 0.5],
                                [-0.5],
                                [ 0.5]]))

    batch_ctr = 0
    batch_to = 0
    report_state = True
    done = False

    for i in range(100):
        txm = dict()
        
        # Check for new instructions
        while jc.poll():
            rxm = jc.recv()
            print(rxm)
            for k,v in rxm.items():
                if k == 'eta':
                    net.eta = v
                elif k == 'batch to':
                    batch_to = v
                elif k == 'tell state':
                    report_state = True
                elif k == 'shutdown':
                    done = True
        
        # Report states if it's the right batch phase, or if asked to
        report_state = report_state or batch_ctr % 4 == 0 and last_state_report_at_batch < batch_ctr

        if report_state:
            txm['eta'] = [batch_ctr, net.eta]
            txm['sv'] = [batch_ctr, list(float(v) for v in net.state_vector())]
            last_state_report_at_batch = batch_ctr
            report_state = False
            
        # Run a learning step if we aren't at the target number of steps
        if batch_to > batch_ctr:
            loss = net.learn([training_batch])
            batch_ctr += 1
            txm['loss'] = [batch_ctr, loss]
        else:
            time.sleep(0.05)
            
        jc.send(txm)
        if done:
            break

    jc.close()

### The parent initiates

In [6]:
import ipywidgets as widgets

In [7]:
from collections import defaultdict
import time

if __name__ == '__main__':
    pipe = Pipe()
    parent_conn, child_conn = pipe
    jc = JSONConn(parent_conn)
    p = Process(target=f, args=(child_conn,))
    p.start()
    
    jc.send({'batch to': 100})
    rxen = defaultdict(list)
    for i in range(110):        
        if jc.poll():
            try:
                m = jc.recv()
                #print(m)
                for k, v in m.items():
                    #print(f"key is {k}, val is {v}")
                    rxen[k].append(v)
            except EOFError:
                print("sender closed")
                break
        else:
            sleep(0.1)

        try:
            last_batch_rx = rxen['loss'][-1][0]
        except (KeyError, IndexError):
            last_batch_rx = 0
        print(f"{last_batch_rx}", end='\r')

    p.join()


{'batch to': 100}
sender closed


In [8]:
rxen['loss'][-1]

[100, 0.1324095965261941]

---

In [9]:
assert False, "stop here if entering from above"

AssertionError: stop here if entering from above

## UI using `asyncio`

In [ ]:
%gui asyncio

In [ ]:
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [ ]:
from ipywidgets import IntSlider, Output
slider = IntSlider()
out = Output()

In [ ]:
async def f():
    for i in range(10):
        out.append_stdout('did work ' + str(i) + '\n')
        x = await wait_for_change(slider, 'value')
        out.append_stdout('async function continued with value ' + str(x) + '\n')
asyncio.ensure_future(f())

slider

In [ ]:
out

In [ ]:
2+2

In [ ]:
slider.value

In [ ]:
import threading
from IPython.display import display
import ipywidgets as widgets
import time
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

def work(progress):
    total = 100
    for i in range(total):
        time.sleep(0.2)
        progress.value = float(i+1)/total

thread = threading.Thread(target=work, args=(progress,))
display(progress)
thread.start()

In [ ]:
i=2

In [ ]:
i+1

In [ ]:
import multiprocessing
multiprocessing.cpu_count()